In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import datetime
import os
import xml.etree.ElementTree as ET

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error

In [12]:
if torch.cuda.is_available():
    print("CUDA is available! Training on GPU...")
else:
    print("CUDA is not available. Training on CPU...")

CUDA is available! Training on GPU...


In [2]:
# Find the overlap
import os

# Function to get filenames from a folder
def get_filenames(folder_path):
    return set(os.listdir(folder_path))

# Replace these paths with your actual folder paths
folder1 = f'C:/Users/username/OneDrive/Desktop/BGprediction/FA_split'
folder2 = f'C:/Users/username/OneDrive/Desktop/BGprediction/ML_split'
folder3 = f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split'
folder4 = f'C:/Users/username/OneDrive/Desktop/BGprediction/FACM_split'

# Get filenames from each folder
filenames1 = get_filenames(folder1)
filenames2 = get_filenames(folder2)
filenames3 = get_filenames(folder3)
filenames4 = get_filenames(folder4)

# Find common filenames across all folders
common_filenames = filenames1 & filenames2 & filenames3 & filenames4


In [11]:
random_filenames = ['1100.csv',
 '870.csv',
 '958.csv',
 '255.csv',
 '933.csv',
 '1718.csv',
 '367.csv',
 '854.csv',
 '95.csv',
 '1287.csv']

Align first, then split, otherwise the order can be strange

# CGM

In [13]:
# # Define the directory path
# train_directory_path = r'C:\Users\username\OneDrive\Desktop\BGprediction\LB_split'  # Use a raw string for paths on Windows

# # List files without their extensions
# train_file_names = [os.path.splitext(file)[0] for file in os.listdir(train_directory_path)
#               if os.path.isfile(os.path.join(train_directory_path, file))]

# # Print the list of file names
# print(train_file_names)

subject = pd.read_csv(f"../LB_split/1100.csv")

In [14]:
def preprocess_t1dexi(path):

    subject = pd.read_csv(path)
    subject['LBDTC'] = pd.to_datetime(subject['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
    # print(subject['date'][0])
    subject.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column

    # # Assuming self.interval_timedelta is set, for example:
    # interval_timedelta = datetime.timedelta(minutes=6)  # Example timedelta of 6 minutes, providing a range for latency

    # # Create a list to store the results
    # res = []

    # # Initialize the first group
    # if not subject.empty:
    #     current_group = [subject.iloc[0]['LBORRES']]
    #     last_time = subject.iloc[0]['LBDTC']

    # # Iterate over rows in DataFrame starting from the second row
    # for index, row in subject.iloc[1:].iterrows():
    #     current_time = row['LBDTC']
    #     if (current_time - last_time) <= interval_timedelta:
    #         # If the time difference is within the limit, add to the current group
    #         current_group.append(row['LBORRES'])
    #     else:
    #         # Otherwise, start a new group
    #         res.append(current_group)
    #         current_group = [row['LBORRES']]
    #     last_time = current_time

    # # Add the last group if it's not empty
    # if current_group:
    #     res.append(current_group)
    
    # # Filter out groups with fewer than 10 glucose readings
    # res = [group for group in res if len(group) >= 10]

    return subject

In [15]:
cgm = preprocess_t1dexi(f"../LB_split/1100.csv")

In [16]:
cgm

,STUDYID,DOMAIN,USUBJID,LBSEQ,LBTESTCD,LBTEST,LBCAT,LBORRES,LBORRESU,LBSTRESC,LBSTRESN,LBSTRESU,LBDTC
7978,T1DEXI,LB,1100,7979,HBA1C,Hemoglobin A1c,DIABETES HISTORY,5.8,%,5.8,5.8,%,2019-12-29 00:00:00
0,T1DEXI,LB,1100,1,GLUC,Glucose,CGM,119.0,mg/dL,119.0,119.0,mg/dL,2020-02-14 00:02:01
1,T1DEXI,LB,1100,2,GLUC,Glucose,CGM,115.0,mg/dL,115.0,115.0,mg/dL,2020-02-14 00:07:01
2,T1DEXI,LB,1100,3,GLUC,Glucose,CGM,112.0,mg/dL,112.0,112.0,mg/dL,2020-02-14 00:12:01
3,T1DEXI,LB,1100,4,GLUC,Glucose,CGM,111.0,mg/dL,111.0,111.0,mg/dL,2020-02-14 00:17:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,T1DEXI,LB,1100,7974,GLUC,Glucose,CGM,211.0,mg/dL,211.0,211.0,mg/dL,2020-03-12 23:37:00
7974,T1DEXI,LB,1100,7975,GLUC,Glucose,CGM,216.0,mg/dL,216.0,216.0,mg/dL,2020-03-12 23:42:00
7975,T1DEXI,LB,1100,7976,GLUC,Glucose,CGM,216.0,mg/dL,216.0,216.0,mg/dL,2020-03-12 23:47:00
7976,T1DEXI,LB,1100,7977,GLUC,Glucose,CGM,218.0,mg/dL,218.0,218.0,mg/dL,2020-03-12 23:52:00


# Meal

## Meal quality check

In [31]:
import os
import pandas as pd

# Path to the directory containing your CSV files
directory_path = '../ML_split'

# List to store filenames with no "NA" in the "MLDOSE" column
valid_files = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if the "MLDOSE" column has no "NA" values
        if df["MLDOSE"].notna().all():
            valid_files.append(filename)


In [32]:
len(valid_files)

101

In [ ]:
def preprocess_t1dexi(path):
    subject = pd.read_csv(path)
    subject['LBDTC'] = pd.to_datetime(subject['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
    # print(subject['date'][0])
    subject.sort_values('LBDTC', inplace=True)  

In [18]:
subject_meal = pd.read_csv(f"../ML_split/870.csv")
subject_meal

,Unnamed: 0,STUDYID,DOMAIN,USUBJID,MLSEQ,MLGRPID,MLTRT,MLCAT,MLDOSE,MLDOSU,MLDOSFRQ,MLDTC,MLDECDCD,MLDNAMVR
0,58153,T1DEXI,ML,870,1,47243,"Almond milk, sweetened",Breakfast,NaN,g,NaN,2020-06-03 08:42:43,11350000.0,FNDDS 2017-2018
1,58154,T1DEXI,ML,870,2,48372,"Almond milk, sweetened",Breakfast,NaN,g,NaN,2020-06-10 08:44:01,11350000.0,FNDDS 2017-2018
2,58155,T1DEXI,ML,870,3,47752,"Almond milk, sweetened, chocolate",Breakfast,NaN,g,NaN,2020-06-06 07:58:30,11350010.0,FNDDS 2017-2018
3,58156,T1DEXI,ML,870,4,48538,"Almond milk, sweetened, chocolate",Breakfast,NaN,g,NaN,2020-06-11 08:23:36,11350010.0,FNDDS 2017-2018
4,58157,T1DEXI,ML,870,5,49142,"Asparagus, cooked, NS as to form, NS as to fat...",Dinner,NaN,g,NaN,2020-06-14 17:28:19,75202000.0,FNDDS 2017-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,58301,T1DEXI,ML,870,149,51110,"Tortilla chips, plain",Evening Snack,NaN,g,NaN,2020-06-26 21:17:08,54401075.0,FNDDS 2017-2018
149,58302,T1DEXI,ML,870,150,47144,"Turkey, prepackaged or deli, luncheon meat",Lunch,NaN,g,NaN,2020-06-02 14:34:33,25230780.0,FNDDS 2017-2018
150,58303,T1DEXI,ML,870,151,49093,"Vegetable combination, including carrots, broc...",Lunch,NaN,g,NaN,2020-06-14 12:40:19,75330100.0,FNDDS 2017-2018
151,58304,T1DEXI,ML,870,152,47990,"Walnuts, honey roasted",Dinner,NaN,g,NaN,2020-06-07 16:42:39,42116100.0,FNDDS 2017-2018


In [26]:
subject_meal["MLDOSE"]

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
148   NaN
149   NaN
150   NaN
151   NaN
152   NaN
Name: MLDOSE, Length: 153, dtype: float64